In [ ]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.events import MouseMove

# For inline display in a notebook
output_notebook()

# Load the CSV file
df = pd.read_csv("tree_datasets.csv")

# Clean column names by stripping whitespace (and any extra characters)
df.columns = [col.strip() for col in df.columns if col.strip() != ""]

# Define the coordinate pairs based on the cleaned header names
pairs = [
    ("Ix", "Illicium verum"),
    ("Mx", "Masson pine"),
    ("Cx", "Chinese fir"),
    ("Sx", "Splash pine")
]

# Create a Bokeh figure
p = figure(title="Tree height distribution in southern China", width=800, height=400, tools="")

# Create a line for each coordinate pair using its own ColumnDataSource
renderers = []
for (xcol, ycol) in pairs:
    # Ensure the columns exist after cleaning
    if xcol in df.columns and ycol in df.columns:
        source = ColumnDataSource(data=dict(x=df[xcol], y=df[ycol]))
        # Use a combined label for the legend
        r = p.line('x', 'y', source=source, line_width=2, alpha=1.0, legend_label=f"{ycol}")
        renderers.append(r)
    else:
        print(f"Column pair ({xcol}, {ycol}) not found in DataFrame.")

# CustomJS callback to change opacity based on mouse proximity to a curve
callback_code = """
// Adjust threshold according to your data's scale
const threshold = 0.1;
let hovered_index = -1;

// Check each renderer's points to see if the mouse is near any point
for (let i = 0; i < renderers.length; i++) {
    const data = renderers[i].data_source.data;
    const xs = data['x'];
    const ys = data['y'];
    let min_dist = Infinity;
    for (let j = 0; j < xs.length; j++) {
        const dx = xs[j] - cb_obj.x;
        const dy = ys[j] - cb_obj.y;
        const dist = Math.sqrt(dx*dx + dy*dy);
        if (dist < min_dist) {
            min_dist = dist;
        }
    }
    if (min_dist < threshold) {
        hovered_index = i;
        break;
    }
}

// Set opacity: the hovered curve remains at 1.0 and others become 0.5
for (let i = 0; i < renderers.length; i++) {
    if (hovered_index === -1) {
        renderers[i].glyph.line_alpha = 1.0;
    } else {
        renderers[i].glyph.line_alpha = (i === hovered_index) ? 1.0 : 0.5;
    }
}
p.change.emit();
"""

# Create and attach the callback
callback = CustomJS(args=dict(renderers=renderers, p=p), code=callback_code)
p.js_on_event(MouseMove, callback)

p.legend.location = "top_left"
p.legend.click_policy = "hide"  # Optional: allows toggling curves on/off via the legend

show(p)
